# TopicBank: Bank Creation Experiment

Dataset: [PostNauka](https://postnauka.ru/) articles.

In [2]:
# General imports

import itertools
import json
import numpy as np
import os
import pandas as pd

from scipy.stats import gaussian_kde
from matplotlib import pyplot as plt

%matplotlib inline

In [3]:
import sys

In [4]:
# Making `topnum` module visible for Python

# TODO
# import sys

sys.path.insert(0, '..')
sys.path.insert(0, '../..')

In [5]:
# Optimal number of topics

from topicnet.cooking_machine import Dataset

from topnum.data.vowpal_wabbit_text_collection import VowpalWabbitTextCollection
from topnum.scores import (
    IntratextCoherenceScore,
    SparsityPhiScore,
    SparsityThetaScore,
    SimpleTopTokensCoherenceScore,
    SophisticatedTopTokensCoherenceScore,
)
from topnum.scores._base_coherence_score import (
    SpecificityEstimationMethod,
    TextType,
    WordTopicRelatednessType
)
from topnum.scores.intratext_coherence_score import ComputationMethod
from topnum.search_methods import TopicBankMethod
from topnum.search_methods.topic_bank.one_model_train_funcs import (
    default_train_func,
    regularization_train_func,
    specific_initial_phi_train_func,
    background_topics_train_func,
)


## Data

In the folder below must reside the necessary data file in .csv format.

In [6]:
DATA_FOLDER_PATH = './data'  # TODO  '/data/datasets'

In [7]:
os.listdir(DATA_FOLDER_PATH)

['AG_News.csv',
 'PostNauka_natural_order.csv',
 'Brown.csv',
 'PScience.csv',
 'Post_Science',
 'Reuters.csv',
 '.ipynb_checkpoints',
 '20NG_natural_order.csv',
 'Watan2004.csv']

In [8]:
DATASET_FOLDER_NAME = 'Watan2004' # 'AG_News' # '20NG_natural_order' # 'Brown' # 'Reuters' # 'Post_Science'

In [9]:
os.listdir(os.path.join(DATA_FOLDER_PATH, DATASET_FOLDER_NAME))

FileNotFoundError: [Errno 2] No such file or directory: './data/Watan2004'

In [10]:
DATASET_FILE_NAME = 'Watan2004.csv' # 'AG_News.csv' # '20NG_natural_order.csv' # 'Brown.csv' # 'Reuters.csv' # 'PScience.csv'

In [11]:
DATASET_FILE_PATH = os.path.join(
    DATA_FOLDER_PATH,
    # DATASET_FOLDER_NAME,
    DATASET_FILE_NAME,
)

Checking if all OK with data, what modalities does the collection have.

In [12]:
! head -n 2 $DATASET_FILE_PATH

id,vw_text,raw_text
"doc_0_00000","doc_0_00000 |@word بكي ا ش ا اعلن صين رسمي كتمل طول قدم ملحم شعري علي مستوي عالم معروف باسم ملحم ملك قصار اثر عثور علي جزئ ناقص وهي عبار الف كلم منقوش علي حجر تماثيل كائن بمعبد جين لونغ سمك ذهبي بمحافظ دانب بمقاطع سيتشو جنوب غرب صين عتبر ملحم قصار طول ملحم شعبي ثقافي عرف عالم تقع 36 مجلد تضم قراب مليوني بيت شعر ضاهي قيم ادبي شهر ملاحم غربي حتي انه طلق علي الياذ شرقي علي غرار ملحم يوناني خالد الياذ اوديسا شاعر هوميروس كتب ملحم قبل الف عام تناول قصة حيا ملك اسطوري قصار لذي هزم قبائل غزو عدو حافظ علي هوي مقاطع تبت جنوب غرب صين طمس تشوي وقد ترجم لغت اصلي لتي كتب وهي منغولي الي عديد لغ عالمي انجليزي فرنسي الماني روسي هندي صيني ياباني ،وتدرس قسم ادب معاهد كلي جامع كثر اربع دول وقد صدر بشا قراب ثلاث الف بحث دراس عديد دوائر اكديمية عالمي كان منظم يونسكو ادرج ضمن مناسب عالمي لتي شار حتفل جميع دول عضء منظم خلال عام ماضي تخليد لتل ملحم فريد باعتبار اثر ثقافي وحيد متبقي حتي يوم رغم مرور كثر الف عام علي صدور","   بكين أ ش أ أعلنت الصين رسميا عن                    

In [13]:
DATASET_INTERNALS_FOLDER_PATH = os.path.join('.', DATASET_FOLDER_NAME + '__internals')

In [14]:
DATASET_INTERNALS_FOLDER_PATH

'./Watan2004__internals'

In [15]:
dataset = Dataset(
    DATASET_FILE_PATH,
    internals_folder_path=DATASET_INTERNALS_FOLDER_PATH
)

In [16]:
os.listdir(DATASET_INTERNALS_FOLDER_PATH)

['dict.dict',
 'new_ppmi_tf_',
 'vocab.txt',
 'cooc_values.json',
 'batches',
 'ppmi_tf_',
 'vw.txt',
 'result']

In [138]:
dataset.get_batch_vectorizer()

artm.BatchVectorizer(data_path="./Watan2004__internals/batches", num_batches=21)

In [139]:
os.listdir(DATASET_INTERNALS_FOLDER_PATH)

['batches', 'vw.txt']

In [140]:
dataset._data.shape

(20291, 3)

In [141]:
dataset._data.head()

,id,vw_text,raw_text
id,,,
doc_0_00000,doc_0_00000,doc_0_00000 |@word بكي ا ش ا اعلن صين رسمي كتم...,بكين أ ش أ أعلنت الصين رسميا عن ...
doc_0_00001,doc_0_00001,doc_0_00001 |@word حاور دمشق حيد تاج ادب روائي...,حاوره في دمشق وحيد تاجا: الأدب الروائي.. ...
doc_0_00002,doc_0_00002,doc_0_00002 |@word كتب سالم رحبي رعي معالي دكت...,كتب سالم الرحبي: ترعى معالي الدكتورة ...
doc_0_00003,doc_0_00003,doc_0_00003 |@word صلال سعيد شاطر رعي سعاد شيخ...,صلالة من سعيد الشاطر: رعى سعادة ...
doc_0_00004,doc_0_00004,doc_0_00004 |@word باريس كون وقت لذي عالج ابن ...,باريس كونا: في الوقت الذي يعالج فيه ابنه ...


In [20]:
len(dataset.get_vw_document('32230.txt').loc['32230.txt', 'vw_text'].split())

710

In [21]:
dataset.get_vw_document('32230.txt').loc['32230.txt', 'vw_text']

'|@text faq наука третий класс факт эксперимент результат полностью предсказывать теория начало известно со школьный скамья задача любой естественный наука число физика открытие новый знание природа разработка принципиально новый технология число первый относиться работа ньютон максвелл эйнштейн фейнман пример второй являться изобретение транзистор лазер технология являться результат целенаправленный научный изыскание входить каждый дом перевертывать мир последний год мировой научный мода входить совершенно другой класс работа исследование известный явление открывать дорога новый неожиданный интерпретация ломать общепринятый стереотип иллюстрировать новый научный идея траектория фотон post idв конец авторитетный научнопопулярный английский издание physics world опубликовывать рейтинг самый выдающийся достижение физика первый место список стоять работа проводить канадский университет торонто руководство профессор эфраим стайнберг суть работа согласно принцип неопределенность гейзенберг 

## Coocs

The notebook [Making-Decorrelation-and-Topic-Selection-Friends.ipynb](https://github.com/machine-intelligence-laboratory/TopicNet/blob/master/topicnet/demos/Making-Decorrelation-and-Topic-Selection-Friends.ipynb) contains a bit more explanation and references concerning cooccurrences computation in ARTM library.

In [22]:
COOC_DATA_FOLDER_PATH = os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'cooc')

In [23]:
if os.path.isdir(COOC_DATA_FOLDER_PATH):
    print(os.listdir(COOC_DATA_FOLDER_PATH))

In [18]:
cooc_values_file_path = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'cooc_values.json',
)

In [19]:
if os.path.isfile(cooc_values_file_path):
    print(
        json.loads(open(cooc_values_file_path, 'r').read())[:20]
    )

[[['هرير', 'تمني'], 1.82768], [['تمني', 'هرير'], 1.82768], [['هرير', 'كبر'], 0.581746], [['كبر', 'هرير'], 0.581746], [['هرير', 'داود'], 2.47035], [['داود', 'هرير'], 2.47035], [['هرير', 'تلق'], 2.12148], [['تلق', 'هرير'], 2.12148], [['هرير', 'بدو'], 0.443265], [['بدو', 'هرير'], 0.443265], [['هرير', 'سعيد'], 0.706545], [['سعيد', 'هرير'], 0.706545], [['هرير', 'ملائك'], 2.00346], [['ملائك', 'هرير'], 2.00346], [['هرير', 'اني'], 1.07549], [['اني', 'هرير'], 1.07549], [['هرير', 'تفتيش'], 2.54284], [['تفتيش', 'هرير'], 2.54284], [['هرير', 'مر'], 1.10353], [['مر', 'هرير'], 1.10353]]


In [20]:
if not os.path.isfile(cooc_values_file_path):
    cooc_values = dict()
else:
    raw_cooc_values = json.loads(open(cooc_values_file_path, 'r').read())

    cooc_values = {
        tuple(d[0]): d[1] for d in raw_cooc_values
    }

In [21]:
len(list(cooc_values.items()))

3514220

In [22]:
print(list(cooc_values.items())[:10])

[(('هرير', 'تمني'), 1.82768), (('تمني', 'هرير'), 1.82768), (('هرير', 'كبر'), 0.581746), (('كبر', 'هرير'), 0.581746), (('هرير', 'داود'), 2.47035), (('داود', 'هرير'), 2.47035), (('هرير', 'تلق'), 2.12148), (('تلق', 'هرير'), 2.12148), (('هرير', 'بدو'), 0.443265), (('بدو', 'هرير'), 0.443265)]


In [23]:
COOC_VALUES = cooc_values

In [24]:
len(COOC_VALUES)

3514220

In [25]:
COOC_VALUES = {
    k: v for i, (k, v) in enumerate(COOC_VALUES.items()) if i % 2 == 0
}

In [26]:
len(COOC_VALUES)

1757110

In [24]:
del cooc_values

In [27]:
median_cooc = np.percentile(list(COOC_VALUES.values()), 60)

In [28]:
median_cooc

0.9181784

In [29]:
COOC_VALUES = {
    k: v for i, (k, v) in enumerate(COOC_VALUES.items()) if v >= median_cooc
}

In [30]:
len(COOC_VALUES)

702844

In [158]:
print(list(COOC_VALUES.items())[:10])

[(('states', 'section'), 0.960109), (('states', 'basis'), 0.329515), (('states', 'defend'), 1.82707), (('states', 'existence'), 0.0594168), (('states', 'force'), 1.10638), (('states', 'express'), 1.33526), (('states', 'west'), 0.530156), (('states', 'learn'), 0.0611878), (('states', 'additional'), 0.428594), (('states', 'proper'), 1.72512)]


In [180]:
from topicnet.cooking_machine.models import TopicModel
import artm

In [183]:
a = artm.ARTM(10)
a.initialize(dataset.get_dictionary())

In [184]:
tm = TopicModel(a)

In [185]:
tm._fit(dataset.get_batch_vectorizer(), 1)

In [186]:
tm.get_phi().head()

topic_0       topic_1       topic_2       topic_3  \
modality token                                                                 
@word    كرديو        7.634554e-08  1.435572e-08  3.222526e-07  3.305830e-08   
         سيانيد       1.079096e-08  1.201125e-07  5.929413e-09  2.212184e-07   
         اتفاي        2.399903e-07  2.337123e-07  4.510908e-07  1.028343e-07   
         اختيارالكلم  4.796336e-08  1.434721e-07  1.377251e-07  5.181442e-08   
         لنساهل       3.547773e-08  4.450235e-08  1.222880e-07  5.289273e-08   

                           topic_4       topic_5       topic_6       topic_7  \
modality token                                                                 
@word    كرديو        7.221951e-08  9.443286e-08  1.186938e-07  1.482633e-07   
         سيانيد       1.473765e-07  1.852361e-08  7.600426e-08  1.382410e-07   
         اتفاي        2.932988e-07  1.215687e-08  1.628408e-07  1.515525e-07   
         اختيارالكلم  2.077898e-07  2.645868e-07  2.835146e-09  4.897009e-08   
         لنساهل       1.177817e-07  3.532562e-07  2.064689e-09  1.368329e-08   

                           topic_8       topic_9  
modality token                                    
@word    كرديو        5.519292e-08  1.494534e-07  
         سيانيد       1.181957e-07  3.563748e-07  
         اتفاي        5.858392e-07  2.536662e-08  
         اختيارالكلم  1.278964e-07  8.651309e-08  
         لنساهل       2.581789e-07  1.469993e-07

In [187]:
del tm
del a

In [188]:
import tqdm

In [335]:
lengths = []

for d in tqdm.tqdm(dataset._data.index, total=len(dataset._data.index), file=sys.stdout):
    lengths.append(
        len(dataset.get_vw_document(d).loc[d, 'vw_text'].split())
    )

  0%|          | 486/127600 [00:13<56:51, 37.26it/s]  


KeyboardInterrupt: 

In [31]:
lengths = dataset._data['vw_text'].apply(lambda text: len(text.split()))

In [32]:
mean_length = sum(lengths) / dataset._data.shape[0]

In [33]:
median_length = np.median(lengths)

In [ ]:
# PN

mean_length

In [101]:
median_length

461.5

In [154]:
461.5 * 50

23075.0

In [108]:
# Reuters

mean_length

96.72923618835743

In [109]:
median_length

70.0

In [155]:
300 * 70

21000

In [33]:
# Brown

mean_length

1077.038

In [34]:
median_length

1073.0

In [35]:
1073.0 * 20

21460.0

In [162]:
# 20 NG

mean_length

139.84532526796136

In [163]:
median_length

75.0

In [164]:
300 * 75

22500

In [30]:
# AG News

mean_length

29.167257053291536

In [31]:
median_length

29.0

In [34]:
800 * 29

23200

In [32]:
23000 / 29

793.1034482758621

In [32]:
# Watan

mean_length

422.5294958355921

In [33]:
median_length

292.0

In [34]:
23000 / median_length

78.76712328767124

In [38]:
p_25 = np.percentile(lengths, 25)
p_75 = np.percentile(lengths, 75)

In [197]:
p_75

524.0

In [198]:
p_25

174.0

In [199]:
median_length * 20

5840.0

In [39]:
all_documents = [
    d for i, d in enumerate(dataset._data.index)
    if (
        lengths[i] <= p_75
        and
        lengths[i] >= p_25
    )
]

In [40]:
del lengths

In [41]:
len(all_documents)

10202

In [210]:
dataset._data.index.nunique(), dataset._data.shape

(20291, (20291, 3))

In [88]:
seed = 42 # 0 # 42 # 11221963
random = np.random.RandomState(seed)

In [89]:
old_test_documents = test_documents

In [90]:
num_test_documents = 80 # 800 # 300 # 20 # 300 # 50
test_documents = random.choice(all_documents, size=num_test_documents, replace=False)

In [91]:
median_length * num_test_documents

23360.0

In [92]:
len(test_documents)

80

In [93]:
len(set(test_documents))

80

In [94]:
test_documents[:10]

array(['doc_4_00214', 'doc_5_02707', 'doc_0_02199', 'doc_2_01677',
       'doc_6_03308', 'doc_6_01353', 'doc_1_03270', 'doc_6_03218',
       'doc_1_04240', 'doc_1_03776'], dtype='<U11')

In [95]:
test_documents = list(test_documents)

In [96]:
len(set(old_test_documents).intersection(set(test_documents)))

1

In [97]:
set(old_test_documents).intersection(set(test_documents))

{'doc_2_01908'}

## Scores (for Topics and Models)

In [98]:
WINDOW = 20
NUM_TOP_WORDS = 20
MAX_NUM_OUT_WORDS = 5

VERBOSE = False

In [99]:
# Default scores in Topic Bank

main_topic_score = IntratextCoherenceScore(
    name='intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none',
    data=dataset,
    documents=test_documents,
    text_type=TextType.VW_TEXT,
    computation_method=ComputationMethod.SEGMENT_WEIGHT,
    word_topic_relatedness=WordTopicRelatednessType.PWT,
    specificity_estimation=SpecificityEstimationMethod.NONE,
    max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
    window=WINDOW,
    verbose=VERBOSE,
)

other_topic_scores = [
    SophisticatedTopTokensCoherenceScore(
        name='top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none',
        data=dataset,
        documents=test_documents,
        text_type=TextType.VW_TEXT,
        word_topic_relatedness=WordTopicRelatednessType.PWT,
        specificity_estimation=SpecificityEstimationMethod.NONE,
        num_top_words=NUM_TOP_WORDS,
        window=WINDOW,
        verbose=VERBOSE,
    )
]

In [100]:
# Other coherence scores variations

text_type_ids = {
    TextType.VW_TEXT: 'vw',
}
computation_method_ids = {
    ComputationMethod.SEGMENT_WEIGHT: 'seg_weight',
    ComputationMethod.SEGMENT_LENGTH: 'seg_length',
    ComputationMethod.SUM_OVER_WINDOW: 'sow',
}
word_topic_relatedness_type_ids = {
    WordTopicRelatednessType.PWT: 'pwt',
    WordTopicRelatednessType.PTW: 'ptw',
}
specificity_estimation_method_ids = {
    SpecificityEstimationMethod.NONE: 'none',
    SpecificityEstimationMethod.AVERAGE: 'av',
    SpecificityEstimationMethod.MAXIMUM: 'max',
}


param_combinations_intratext = list(
    itertools.product(
        text_type_ids,
        computation_method_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_intratext.remove(
    (
        TextType.VW_TEXT,
        ComputationMethod.SEGMENT_WEIGHT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)

param_combinations_top_tokens = list(
    itertools.product(
        text_type_ids,
        word_topic_relatedness_type_ids,
        specificity_estimation_method_ids,
    )
)
param_combinations_top_tokens.remove(
    (
        TextType.VW_TEXT,
        WordTopicRelatednessType.PWT,
        SpecificityEstimationMethod.NONE
    )
)


for param_combination in param_combinations_intratext:
    (text_type,
     computation_method,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'intratext_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__cm_{computation_method_ids[computation_method]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    other_topic_scores.append(
        IntratextCoherenceScore(
            name=name,
            data=dataset,
            documents=test_documents,
            text_type=text_type,
            computation_method=computation_method,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            max_num_out_of_topic_words=MAX_NUM_OUT_WORDS,
            window=WINDOW,
            verbose=VERBOSE,
        )
    )


for param_combination in param_combinations_top_tokens:
    (text_type,
     word_topic_relatedness,
     specificity_estimation) = param_combination

    name = (
        f'top_tokens_coherence_score'
        f'__tt_{text_type_ids[text_type]}'
        f'__wtrt_{word_topic_relatedness_type_ids[word_topic_relatedness]}'
        f'__sem_{specificity_estimation_method_ids[specificity_estimation]}'
    )

    other_topic_scores.append(
        SophisticatedTopTokensCoherenceScore(
            name=name,
            data=dataset,
            documents=test_documents,
            text_type=text_type,
            word_topic_relatedness=word_topic_relatedness,
            specificity_estimation=specificity_estimation,
            #word_cooccurrences=COOC_VALUES2,  # TODO!
            num_top_words=NUM_TOP_WORDS,
            window=WINDOW,
            verbose=VERBOSE,
        )
    )

In [101]:
len(other_topic_scores)

23

In [102]:
len(COOC_VALUES)

702844

In [103]:
# Another implementation of top-tokens-based coherence

# param_combinations_other_top_tokens = list(
#     itertools.product([True, False], ['median', 'mean'], [None, 1e-7])
# )

param_combinations_other_top_tokens = list(
    itertools.product([True], ['median'], [1e-7])
)

if len(COOC_VALUES) > 0:  # with pre-computed coocs
    for param_combination in param_combinations_other_top_tokens:
        (kernel,
         average,
         active_topic_threshold) = param_combination

        name = (
            f'top_tokens_coherence_other_implementation_score'
            f'__ker_{kernel}'
            f'__av_{average}'
            f'__att_{active_topic_threshold}'
        )

        other_topic_scores.append(
            SimpleTopTokensCoherenceScore(
                name=name,
                data=dataset,
                cooccurrence_values=COOC_VALUES,
                num_top_tokens=20,
                kernel=kernel,
                average=average,
                active_topic_threshold=active_topic_threshold,
            )
        )

In [104]:
len(other_topic_scores)

24

In [105]:
# Default scores in Topic Bank

other_scores = [
    SparsityPhiScore(
        name='sparsity_phi_score'
    ),
    SparsityThetaScore(
        name='sparsity_theta_score'
    ),
]

## Bank Creation

In [106]:
# Default train func

TRAIN_FUNCS = default_train_func

In [221]:
os.makedirs(os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'result'))

In [107]:
os.path.isdir(os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'result', f'bank__{seed}'))

False

In [108]:
os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'result', f'bank__{seed}')

'./Watan2004__internals/result/bank__42'

In [109]:
os.makedirs(os.path.join(DATASET_INTERNALS_FOLDER_PATH, 'result', f'bank__{seed}'))

In [110]:
seed

42

In [115]:
# TODO: use Holdout Perplexity as Stop score

start_model_number = 1  # TODO

optimizer = TopicBankMethod(
    data = dataset,
    min_df_rate = 0.025,
    max_df_rate = 0.8,

    main_topic_score = main_topic_score,
    other_topic_scores = other_topic_scores,
    other_scores = other_scores,
    
    documents = test_documents,

    start_model_number = start_model_number,
    max_num_models = 20,
    one_model_num_topics = 100,
    num_fit_iterations = 100,
    topic_score_threshold_percentile = 90,

    save_bank = True,
    save_model_topics = True,
    
    save_file_path = os.path.join(
        DATASET_INTERNALS_FOLDER_PATH, 'result', f'search_result__{seed}.json'),
    bank_folder_path = os.path.join(
        DATASET_INTERNALS_FOLDER_PATH, 'result', f'bank__{seed}'),

    train_funcs = TRAIN_FUNCS,
    
    verbose = True,
)

../topnum/search_methods/topic_bank/topic_bank_method.py:257: UserWarning: File "./Watan2004__internals/result/search_result__42.json" already exists. Loading
  warnings.warn(f'File "{save_file_path}" already exists. Loading')


In [116]:
optimizer._result.keys()

dict_keys(['optimum', 'optimum_std', 'bank_scores', 'bank_topic_scores', 'model_scores', 'model_topic_scores', 'num_bank_topics', 'num_model_topics'])

In [117]:
optimizer._result['num_bank_topics']

[10]

In [118]:
len(optimizer._result['num_bank_topics'])

1

In [76]:
optimizer._save_file_path

'./Watan2004__internals/result/search_result__11221963.json'

In [87]:
len(optimizer._result['num_bank_topics'])

20

In [127]:
optimizer._topic_bank._path

'./AG_News__internals/result/bank__0'

In [84]:
optimizer._topic_bank.view_topics().head()

topic_0   topic_1  topic_2   topic_3       topic_4  \
@word تمني  0.000000e+00  0.000000      0.0  0.000000  0.000000e+00   
      جوز   0.000000e+00  0.000000      0.0  0.000000  0.000000e+00   
      ربط   7.607563e-07  0.000000      0.0  0.000007  0.000000e+00   
      خط    7.317736e-13  0.000000      0.0  0.000000  0.000000e+00   
      كبر   3.088602e-11  0.000006      0.0  0.002187  7.476166e-13   

                 topic_5       topic_6       topic_7       topic_8  \
@word تمني  0.000000e+00  0.000000e+00  6.471352e-14  0.000000e+00   
      جوز   0.000000e+00  4.923269e-04  0.000000e+00  0.000000e+00   
      ربط   1.301171e-07  4.888294e-14  0.000000e+00  3.320361e-10   
      خط    0.000000e+00  0.000000e+00  0.000000e+00  3.579498e-10   
      كبر   3.381091e-08  2.038143e-10  7.529581e-04  9.068673e-07   

                 topic_9  
@word تمني  5.845980e-05  
      جوز   0.000000e+00  
      ربط   0.000000e+00  
      خط    0.000000e+00  
      كبر   1.399392e-10

In [63]:
optimizer._topic_bank.view_topics().shape

(1913, 10)

In [85]:
optimizer._topic_bank.view_topics()['topic_2'].sort_values(ascending=False)[:10]

@word  و       0.550311
       لذي     0.023649
       شيخ     0.019949
       فقد     0.006017
       صدر     0.006008
       قائم    0.005285
       رحم     0.005047
       حكم     0.004551
       وفي     0.004416
       سعيد    0.004405
Name: topic_2, dtype: float64

Fulfilling the search:

In [66]:
all_scores = [main_topic_score] + other_topic_scores

In [73]:
len(all_scores)

25

In [74]:
for i, s in enumerate(all_scores):
    print(f'{i + 1:2} {s._name}')

 1 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none
 2 top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none
 3 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_av
 4 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_max
 5 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_none
 6 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_av
 7 intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_max
 8 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_none
 9 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_av
10 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_max
11 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_ptw__sem_none
12 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_ptw__sem_av
13 intratext_coherence_score__tt_vw__cm_seg_length__wtrt_ptw__sem_max
14 intratext_coherence_score__tt_vw__cm_sow__wtrt_pwt__sem_none
15 intratext_coherence_score__tt_vw__cm

In [71]:
len(all_scores[2]._word_cooccurrences)

465882

In [80]:
len(COOC_VALUES2)

466196

In [ ]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [31:34<02:00, 120.68s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [31:33<02:00, 120.57s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [31:42<02:01, 121.86s/it]

...


 84%|████████▍ | 21/25 [25:09<07:17, 109.40s/it]

 96%|█████████▌| 24/25 [31:29<02:00, 120.79s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [32:06<02:01, 121.35s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 24%|██▍       | 6/25 [05:58<16:25, 51.87s/it]

In [78]:
1

1

In [198]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [26:24<01:51, 111.51s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [26:26<01:51, 111.04s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [26:26<01:51, 111.19s/it]

...


 96%|█████████▌| 24/25 [26:23<01:51, 111.23s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [26:22<01:50, 110.97s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



100%|██████████| 20/20 [9:15:39<00:00, 1666.95s/it]
CPU times: user 11h 1min 59s, sys: 1h 12min 32s, total: 12h 14min 32s
Wall time: 9h 15min 39s


In [ ]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [33:32<02:04, 124.56s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [33:20<02:03, 123.67s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [33:23<02:04, 124.71s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 76%|███████▌  | 19/25 [24:36<10:43, 107.19s/it]

In [65]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [30:08<01:49, 109.34s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [29:41<01:49, 109.01s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [31:00<02:05, 125.83s/it]

...


 96%|█████████▌| 24/25 [31:57<01:52, 112.94s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [31:30<01:59, 119.87s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



100%|██████████| 20/20 [11:28:02<00:00, 2064.15s/it]
CPU times: user 14h 35min 25s, sys: 1h 46min 6s, total: 16h 21min 31s
Wall time: 11h 28min 3s


In [68]:
bank_topics = optimizer._topic_bank.view_topics()

In [85]:
bank_topics.shape

(2514, 15)

In [69]:
bank_topics.head()

topic_0  topic_1       topic_2  \
@default_class както                  0.0  0.00000  5.497794e-11   
               гравитационный         0.0  0.00000  0.000000e+00   
               жена                   0.0  0.00139  0.000000e+00   
               продолжительность      0.0  0.00000  0.000000e+00   
               одновременно           0.0  0.00075  5.159791e-06   

                                       topic_3       topic_4       topic_5  \
@default_class както              0.000000e+00  1.985204e-08  2.223492e-14   
               гравитационный     0.000000e+00  0.000000e+00  0.000000e+00   
               жена               0.000000e+00  0.000000e+00  1.217113e-04   
               продолжительность  3.470690e-15  0.000000e+00  0.000000e+00   
               одновременно       7.805681e-09  3.031660e-06  5.124066e-09   

                                   topic_6       topic_7   topic_8   topic_9  \
@default_class както              0.000009  0.000000e+00  0.000000  0.000000   
               гравитационный     0.000000  0.000000e+00  0.000000  0.022702   
               жена               0.000000  3.405896e-15  0.000000  0.000000   
               продолжительность  0.000000  4.282006e-04  0.000000  0.000000   
               одновременно       0.000146  8.667931e-13  0.000935  0.000564   

                                      topic_10  topic_11  topic_12  \
@default_class както              0.000000e+00       0.0       0.0   
               гравитационный     0.000000e+00       0.0       0.0   
               жена               0.000000e+00       0.0       0.0   
               продолжительность  3.474610e-08       0.0       0.0   
               одновременно       1.033011e-07       0.0       0.0   

                                      topic_13      topic_14  
@default_class както              2.414336e-06  4.583967e-11  
               гравитационный     0.000000e+00  0.000000e+00  
               жена               1.091175e-13  8.552548e-05  
               продолжительность  0.000000e+00  0.000000e+00  
               одновременно       2.076379e-05  1.407077e-09

In [83]:
bank_topics['topic_14'].sort_values(ascending=False)[:20]

@default_class  век             0.229964
                xx              0.053898
                xix             0.051478
                начало          0.043227
                первый          0.038318
                конец           0.035413
                второй          0.019743
                середина        0.018641
                время           0.017661
                половина        0.017191
                xviii           0.013792
                классический    0.013726
                xvii            0.012661
                возникать       0.011614
                эпоха           0.010723
                хх              0.010398
                новый           0.010231
                столетие        0.008743
                образ           0.008709
                период          0.008326
Name: topic_14, dtype: float64

In [67]:
optimizer._topic_bank.view_topic_scores()

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14
kernel_size,229.000000,408.000000,102.000000,157.000000,235.000000,302.000000,345.000000,347.000000,283.000000,391.000000,338.000000,59.000000,121.000000,407.000000,214.000000
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_none,0.129687,0.123335,0.468014,0.249535,0.211076,0.246103,0.084003,0.103003,0.149715,0.085349,0.102518,0.177264,0.116741,0.102491,0.099711
top_tokens_coherence_score__tt_vw__wtrt_pwt__sem_none,1.133654,0.667880,1.629793,0.835258,0.835292,0.806799,0.434416,0.921406,0.700194,0.409935,0.473712,0.614014,0.583990,0.500487,0.975982
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_av,0.127684,0.121866,0.462997,0.246569,0.207621,0.243621,0.082943,0.101184,0.147921,0.084183,0.101309,0.173868,0.113667,0.101379,0.097300
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_pwt__sem_max,0.120821,0.114016,0.462851,0.245817,0.116343,0.245570,0.079723,0.083172,0.143387,0.075810,0.096531,0.103924,0.100431,0.100097,0.080213
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_none,0.463680,0.783485,0.763705,0.481053,0.594415,0.914586,0.645213,0.466021,0.734765,0.765880,0.903625,0.845633,0.492089,0.642143,0.650835
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_av,0.452750,0.771956,0.753009,0.470381,0.583590,0.903922,0.633082,0.455416,0.723801,0.753463,0.890064,0.830870,0.479616,0.631021,0.636409
intratext_coherence_score__tt_vw__cm_seg_weight__wtrt_ptw__sem_max,0.385764,0.677018,0.718970,0.418170,0.345956,0.879365,0.549258,0.345888,0.661878,0.648797,0.772138,0.636360,0.417972,0.574683,0.525002
intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_none,1.092971,1.152834,1.069599,1.067143,1.082481,1.066313,1.213043,1.060516,1.096398,1.241758,1.356061,1.476266,1.247272,1.112245,1.442545
intratext_coherence_score__tt_vw__cm_seg_length__wtrt_pwt__sem_av,1.092971,1.152834,1.069599,1.067143,1.082481,1.066313,1.213043,1.060516,1.096398,1.241758,1.356061,1.476266,1.247272,1.112245,1.442545


In [73]:
list(COOC_VALUES2.keys())[:1]

[(('@default_class', 'както'), ('@default_class', 'понимать'))]

In [75]:
COOC_VALUES2[(('@default_class', 'понимать'), ('@default_class', 'както'))]

KeyError: (('@default_class', 'понимать'), ('@default_class', 'както'))

In [82]:
list(COOC_VALUES2.keys())[:1]

[(('@default_class', 'както'), ('@default_class', 'понимать'))]

In [81]:
COOC_VALUES2[(('@default_class', 'нервный'), ('@default_class', 'растение'))]

KeyError: (('@default_class', 'нервный'), ('@default_class', 'растение'))

In [ ]:
%%time

optimizer.search_for_optimum(dataset)

 96%|█████████▌| 24/25 [05:22<00:19, 19.23s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [05:17<00:19, 19.28s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [05:17<00:19, 19.30s/it]

...


 96%|█████████▌| 24/25 [05:30<00:19, 19.42s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 96%|█████████▌| 24/25 [05:20<00:19, 19.54s/it]

../topnum/scores/simple_toptok_coherence_score.py:120: UserWarning: The parameter `documents` is not used by SimpleTopTokensCoherenceScore (it is kept for compatibility purposes)
  'The parameter `documents` is not used by SimpleTopTokensCoherenceScore'



 68%|██████▊   | 17/25 [03:04<02:01, 15.22s/it]

In [59]:
%%time

optimizer.search_for_optimum(dataset)

  0%|          | 0/20 [1:38:22<?, ?it/s]


NotImplementedError: 

In [ ]:
%%time

optimizer.search_for_optimum(dataset)

 72%|███████▏  | 18/25 [1:01:25<34:05, 292.15s/it]

In [185]:
%%time

optimizer.search_for_optimum(dataset)

  0%|          | 0/20 [1:21:46<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
! ls $optimizer._topic_bank._path

model_0__phi.bin	   model_4__phi.bin	      model_8__phi.bin
model_0__topic_scores.bin  model_4__topic_scores.bin  model_8__topic_scores.bin
model_1__phi.bin	   model_5__phi.bin	      model_9__phi.bin
model_1__topic_scores.bin  model_5__topic_scores.bin  model_9__topic_scores.bin
model_2__phi.bin	   model_6__phi.bin	      topics.bin
model_2__topic_scores.bin  model_6__topic_scores.bin  topic_scores.bin
model_3__phi.bin	   model_7__phi.bin
model_3__topic_scores.bin  model_7__topic_scores.bin


## Appendix. Making vocab.txt for Computing Coocs

In [182]:
! rm $DATASET_INTERNALS_FOLDER_PATH/dict.dict

In [153]:
dictionary = dataset.get_dictionary()

In [154]:
dictionary

artm.Dictionary(name=19a09c5f-e84a-43e7-bbe1-de8dde814c40, num_entries=110587)

In [155]:
dataset._data.shape

(20291, 3)

In [156]:
# PN: dictionary.filter(min_df_rate=0.04, max_df_rate=0.5)
# Reuters: dictionary.filter(min_df_rate=0.01, max_df_rate=0.5)
# Brown: dictionary.filter(min_df_rate=0.02, max_df_rate=0.5)
# 20 NG: dictionary.filter(min_df_rate=0.01, max_df_rate=0.5)

# AG dictionary.filter(min_df_rate=0.002, max_df_rate=0.5)

# Watan2004
dictionary.filter(min_df_rate=0.01, max_df_rate=0.5)

print(dictionary)

artm.Dictionary(name=19a09c5f-e84a-43e7-bbe1-de8dde814c40, num_entries=4062)


In [157]:
dictionary_file_path = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'dict.dict',
)

dictionary.save_text(dictionary_file_path)

In [158]:
dictionary

artm.Dictionary(name=19a09c5f-e84a-43e7-bbe1-de8dde814c40, num_entries=4062)

In [159]:
lines = open(dictionary_file_path, 'r').readlines()

In [160]:
len(lines)

4064

In [161]:
lines[:3]

['name: 19a09c5f-e84a-43e7-bbe1-de8dde814c40 num_items: 20291\n',
 'token, class_id, token_value, token_tf, token_df\n',
 'هرير, @word, 6.949519593035802e-05, 593.0, 417.0\n']

In [162]:
lines[-3:]

['معالي, @word, 0.0005515082739293575, 4706.0, 1598.0\n',
 'طريق, @word, 0.0010923519730567932, 9321.0, 4924.0\n',
 'سكن, @word, 6.129171379143372e-05, 523.0, 420.0\n']

In [163]:
vocab_text = ''

for line in lines[2:]:
    token, modality, _, _, _ = line.strip().split(', ')
    vocab_text += f'{token} {modality}\n'

In [164]:
vocab_file_path = os.path.join(
    DATASET_INTERNALS_FOLDER_PATH,
    'vocab.txt',
)

In [165]:
with open(vocab_file_path, 'w') as f:
    f.write(vocab_text)

In [ ]:
! ~/Workspace/bigartm/build/bin/bigartm \
    -c vw.txt \
    -v vocab.txt \
    --cooc-window 10 \
    --cooc-min-tf 2 \
    --write-cooc-tf cooc_tf_ \
    --cooc-min-df 2 \
    --write-cooc-df cooc_df_ \
    --write-ppmi-tf ppmi_tf_ \
    --write-ppmi-df ppmi_df_

In [166]:
def transform_coocs_file(source_file_path, target_file_path, vocab_file_path):
    """
    source_file is assumed to be either ppmi_tf_ or ppmi_df_ 
    """
    num_times_to_log = 10

    vocab = open(vocab_file_path, 'r').readlines()
    vocab = [l.strip().split()[0] for l in vocab]
    
    cooc_values = dict()
    word_word_value_triples = set()
    
    lines = open(source_file_path, 'r').readlines()
    
    for i, l in enumerate(lines):
        if i % (len(lines) // num_times_to_log) == 0:
            print(f'{i:6d} lines out of {len(lines)}')
        
        l = l.strip()
        words = l.split()
        words = words[1:]  # exclude modality
        anchor_word = words[0]
        
        other_word_values = words[1:]
        
        for word_and_value in other_word_values:
            other_word, value = word_and_value.split(':')
            value = float(value)
            
            cooc_values[(anchor_word, other_word)] = value
            cooc_values[(other_word, anchor_word)] = value  # if assume cooc values to be symmetric
            
            word_word_value_triples.add(
                tuple([
                    tuple(sorted([
                        vocab.index(anchor_word),
                        vocab.index(other_word)
                    ])),
                    value
                ])
            )
    
    new_text = ''
    
    for (w1, w2), v in word_word_value_triples:
        new_text += f'{w1} {w2} {v}\n'
    
    with open(target_file_path, 'w') as f:
        f.write(''.join(new_text))
    
    return cooc_values

In [167]:
def prefix_path(p):
    return os.path.join(DATASET_INTERNALS_FOLDER_PATH, p)

In [168]:
COOC_VALUES = transform_coocs_file(
    prefix_path('ppmi_tf_'),
    prefix_path('new_ppmi_tf_'),
    prefix_path('vocab.txt')
)

     0 lines out of 4058
   405 lines out of 4058
   810 lines out of 4058
  1215 lines out of 4058
  1620 lines out of 4058
  2025 lines out of 4058
  2430 lines out of 4058
  2835 lines out of 4058
  3240 lines out of 4058
  3645 lines out of 4058
  4050 lines out of 4058


In [169]:
with open(prefix_path('cooc_values.json'), 'w') as f:
    f.write(json.dumps(list(COOC_VALUES.items())))

In [320]:
saved_raw_cooc_values = json.loads(open(prefix_path('cooc_values.json'), 'r').read())

saved_cooc_values = {
    tuple(d[0]): d[1] for d in saved_raw_cooc_values
}

In [321]:
COOC_VALUES = saved_cooc_values

In [170]:
list(COOC_VALUES.items())[:20]

[(('هرير', 'تمني'), 1.82768),
 (('تمني', 'هرير'), 1.82768),
 (('هرير', 'كبر'), 0.581746),
 (('كبر', 'هرير'), 0.581746),
 (('هرير', 'داود'), 2.47035),
 (('داود', 'هرير'), 2.47035),
 (('هرير', 'تلق'), 2.12148),
 (('تلق', 'هرير'), 2.12148),
 (('هرير', 'بدو'), 0.443265),
 (('بدو', 'هرير'), 0.443265),
 (('هرير', 'سعيد'), 0.706545),
 (('سعيد', 'هرير'), 0.706545),
 (('هرير', 'ملائك'), 2.00346),
 (('ملائك', 'هرير'), 2.00346),
 (('هرير', 'اني'), 1.07549),
 (('اني', 'هرير'), 1.07549),
 (('هرير', 'تفتيش'), 2.54284),
 (('تفتيش', 'هرير'), 2.54284),
 (('هرير', 'مر'), 1.10353),
 (('مر', 'هرير'), 1.10353)]

In [171]:
len(COOC_VALUES)

3514220

In [148]:
len(COOC_VALUES)

551060